<a href="https://colab.research.google.com/github/meishihna/GITHUB-COLAB/blob/main/LINE_Notify.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

###**工具**

####定義包

In [ ]:
!pip install FinMind
!pip install --upgrade mplfinance

In [14]:
import pandas as pd
# from Data import getDataYF,getDataFM
import mplfinance as mpf
import matplotlib.pyplot as plt
import os
import numpy as np

In [21]:
import yfinance as yf
import FinMind
from FinMind.data import DataLoader
datapath = "data"
FM = DataLoader()
def getDataYF(prod,st,en):
  data=yf.download(prod,start=st,end=en)
  data.columns=[i.lower() for i in data.columns]
  return data

def getDataFM(prod,st,en):
  FM = DataLoader()
  df = FM.taiwan_stock_daily(stock_id=prod, start_date = st, end_date =en)
## 整理資料格式
  df = df.rename(columns={"date": "Date"})
  df.set_index("Date" , inplace=True)
  df = df.set_index(pd.DatetimeIndex(pd.to_datetime(df.index)))
## backtesting.py 格式
  df1 = df.rename(columns={"open": "Open", "max": "High", "min": "Low", "close": "Close", "Trading_Volume": "Volume"})
## ta-lib 格式
  df2 = df.rename(columns={"max": "high", "min": "low", "Trading_Volume": "Volume"})
  return df1

def getDataFM1(prod,st,en):
  FM = DataLoader()
  df = FM.taiwan_stock_daily(stock_id=prod, start_date = st, end_date =en)
## 整理資料格式
  df = df.rename(columns={"date": "Date"})
  df.set_index("Date" , inplace=True)
  df = df.set_index(pd.DatetimeIndex(pd.to_datetime(df.index)))
## backtesting.py 格式
  df1 = df.rename(columns={"open": "Open", "max": "High", "min": "Low", "close": "Close", "Trading_Volume": "Volume"})
## ta-lib 格式
  df2 = df.rename(columns={"max": "high", "min": "low", "Trading_Volume": "volume"})
  return df2

def ChartCandle(data,addp=[]):
  mcolor = mpf.make_marketcolors(up='r',down='g',inherit = True)
  mstyle = mpf.make_mpf_style(base_mpf_style='yahoo',marketcolors = mcolor)
  mpf.plot(data,addplot = addp,style = mstyle,type = 'candle',volume= True)

def ChartTrade(data,trade=pd.DataFrame(),addp=[],v_enable=True):

  addp = addp.copy()
  data1 = data.copy()

  if trade.shape[0] > 0:
    trade1 = trade.copy()
    # 進場明細
    buy_order_trade = trade1[[2,3]]
    buy_order_trade = buy_order_trade.set_index(2)
    buy_order_trade.columns = ['buy_order']
    buy_order_trade = buy_order_trade.drop_duplicates()
    # 出場明細
    buy_cover_trade = trade1[[4,5]]
    buy_cover_trade = buy_cover_trade.set_index(4)
    buy_cover_trade.columns = ['buy_cover']
    buy_cover_trade = buy_cover_trade.drop_duplicates()

    data1 = pd.concat([data1,buy_order_trade,buy_cover_trade], axis = 1)

    addp.append(mpf.make_addplot(data1['buy_order']
            ,type = 'scatter', color = '#FF4500'
            ,marker = '^',markersize = 50))
    addp.append(mpf.make_addplot(data1['buy_cover']
            ,type = 'scatter',color = '#16982B'
            ,marker = 'v',markersize = 50))
  mcolor = mpf.make_marketcolors(up = 'r',down = 'g',inherit = True)
  mstyle = mpf.make_mpf_style(base_mpf_style='yahoo',marketcolors = mcolor)
  mpf.plot(data1,addplot = addp,style = mstyle,type = 'candle',volume = v_enable)

# 計算交易績效指標
def Performance(trade=pd.DataFrame(),prodtype='ETF'):
    # 如果沒有交易紀錄 則不做接下來的計算
    if trade.shape[0]==0:
        print('沒有交易紀錄')
        return False

    # 交易成本 手續費0.1425%*2 (券商打折請自行計算)
    if prodtype=='ETF':
        cost=0.001 + 0.00285    # ETF稅金 0.1%
    elif prodtype=='Stock':
        cost=0.003 + 0.00285    # 股票的稅金 0.3%
    else:
        return False

    # 將物件複製出來，不影響原本的變數內容
    trade1=trade.copy()
    trade1=trade1.sort_values(4)
    trade1=trade1.reset_index(drop=True)

    # 給交易明細定義欄位名稱
    trade1.columns=['product','bs','order_time','order_price','cover_time','cover_price','order_unit']
    # 計算出每筆的報酬率
    trade1['ret']=(((trade1['cover_price']-trade1['order_price'])/trade1['order_price'])-cost) *trade1['order_unit']

    # 1.	總報酬率：整個回測期間的總報酬率累加
    print('總績效 %s '%( round(trade1['ret'].sum(),4) ))
    # 2.	總交易次數：代表回測的交易筆數
    print('交易次數 %s '%( trade1.shape[0] ))
    # 3.	平均報酬率：簡單平均報酬率（扣除交易成本後）
    print('平均績效 %s '%( round(trade1['ret'].mean(),4) ))
    # 4.    平均持有時間：代表平均每筆交易的持有時間
    onopen_day=(trade1['cover_time']-trade1['order_time']).mean()
    print('平均持有天數 %s 天'%( onopen_day.days ))
    # 判斷是否獲利跟虧損都有績效
    earn_trade=trade1[trade1['ret'] > 0]
    loss_trade=trade1[trade1['ret'] <= 0]
    if earn_trade.shape[0]==0 or loss_trade.shape[0]==0:
        print('交易資料樣本不足(樣本中需要賺有賠的)')
        return False
    # 5.	勝率：代表在交易次數中，獲利次數的佔比（扣除交易成本後）
    earn_ratio=earn_trade.shape[0] / trade1.shape[0]
    print('勝率 %s '%( round(earn_ratio ,2) ))
    # 6.	平均獲利：代表平均每一次獲利的金額（扣除交易成本後）
    avg_earn=earn_trade['ret'].mean()
    print('平均獲利 %s '%( round(avg_earn,4)))
    # 7.	平均虧損：代表平均每一次虧損的金額（扣除交易成本後）
    avg_loss=loss_trade['ret'].mean()
    print('平均虧損 %s '%( round(avg_loss,4) ))
    # 8.	賺賠比：代表平均獲利 / 平均虧損
    odds=abs(avg_earn/avg_loss)
    print('賺賠比 %s '%( round(odds,4) ))
    # 9.	期望值：代表每投入的金額，可能會回報的多少倍的金額
    print('期望值 %s '%( round(((earn_ratio*odds)-(1-earn_ratio)),4) ))
    # 10.	獲利平均持有時間：代表獲利平均每筆交易的持有時間
    earn_onopen_day=(earn_trade['cover_time']-earn_trade['order_time']).mean()
    print('獲利平均持有天數 %s 天'%( earn_onopen_day.days ))
    # 11.	虧損平均持有時間：代表虧損平均每筆交易的持有時間
    loss_onopen_day=(loss_trade['cover_time']-loss_trade['order_time']).mean()
    print('虧損平均持有天數 %s 天'%( loss_onopen_day.days ))

    # 12.	最大連續虧損：代表連續虧損的最大幅度
    tmp_accloss=1
    max_accloss=1
    for ret in trade1['ret'].values:
        if ret <= 0:
            tmp_accloss *= ret
            max_accloss= min(max_accloss,tmp_accloss)
        else:
            tmp_accloss = 1
    print('最大連續虧損',round(max_accloss ,4))

    # 優先計算累計報酬率 並將累計報酬率的初始值改為1 繪圖較容易閱讀
    trade1['acc_ret'] = (1+trade1['ret']).cumprod()
    trade1.loc[-1,'acc_ret'] = 1
    trade1.index = trade1.index + 1
    trade1.sort_index(inplace=True)

    # 13.	最大資金回落：代表資金從最高點回落至最低點的幅度
    trade1['acc_max_cap'] = trade1['acc_ret'].cummax()
    trade1['dd'] = (trade1['acc_ret'] / trade1['acc_max_cap'])
    trade1.loc[trade1['acc_ret'] == trade1['acc_max_cap'] , 'new_high'] = trade1['acc_ret']
    print('最大資金回落',round(1-trade1['dd'].min(),4))

    # 14.	繪製資金曲線圖(用幾何報酬計算)
    ax=plt.subplot(111)
    ax.plot( trade1['acc_ret'] , 'b-' ,label='Profit')
    ax.plot( trade1['dd'] , '-' ,color='#00A600',label='MDD')
    ax.plot( trade1['new_high'] , 'o' ,color='#FF0000',label='Equity high')
    ax.legend()
    plt.show()

    return trade1

def getFMShareHolder(prod,st,en):
  bakfile = '%s_%s_%s_stock_holding_shares_per.csv'%(prod,st,en)

  if os.path.exists(bakfile):
    tmpdata=pd.read_csv(bakfile)
    tmpdata['date']=pd.to_datetime(tmpdata['date'])
    tmpdata=tmpdata.set_index(tmpdata['date'])

  else:
    tmpdata=FM.taiwan_stock_holding_shares_per(stock_id=prod,start_date=st,end_date=en)
    tmpdata['date']=pd.to_datetime(tmpdata['date'])
    tmpdata=tmpdata.set_index(tmpdata['date'])
    tmpdata.drop('date',axis=1,inplace=True)
    tmpdata.to_csv(bakfile)

  return tmpdata

def ShareHolderWeekly(data):
  rs_sh,rs_dt=[],[]

  for dt in data.index.unique():
    data1=data[data.index==dt]

    tmprow = data1.loc[data1['HoldingSharesLevel']=='1-999','percent'].values # 1
    tmprow = np.append(tmprow,data1.loc[data1['HoldingSharesLevel']=='1,000-5,000','percent'].values) # 2
    tmprow = np.append(tmprow,data1.loc[data1['HoldingSharesLevel']=='5,001-10,000','percent'].values) # 3
    tmprow = np.append(tmprow,data1.loc[data1['HoldingSharesLevel']=='10,001-15,000','percent'].values) # 4
    tmprow = np.append(tmprow,data1.loc[data1['HoldingSharesLevel']=='15,001-20,000','percent'].values) # 5
    tmprow = np.append(tmprow,data1.loc[data1['HoldingSharesLevel']=='20,001-30,000','percent'].values) # 6
    tmprow = np.append(tmprow,data1.loc[data1['HoldingSharesLevel']=='30,001-40,000','percent'].values) # 7
    tmprow = np.append(tmprow,data1.loc[data1['HoldingSharesLevel']=='40,001-50,000','percent'].values) # 8
    tmprow = np.append(tmprow,data1.loc[data1['HoldingSharesLevel']=='50,001-100,000','percent'].values) # 9
    tmprow = np.append(tmprow,data1.loc[data1['HoldingSharesLevel']=='100,001-200,000','percent'].values) # 10
    tmprow = np.append(tmprow,data1.loc[data1['HoldingSharesLevel']=='200,001-400,000','percent'].values) # 11
    tmprow = np.append(tmprow,data1.loc[data1['HoldingSharesLevel']=='400,001-600,000','percent'].values) # 12
    tmprow = np.append(tmprow,data1.loc[data1['HoldingSharesLevel']=='600,001-800,000','percent'].values) # 13
    tmprow = np.append(tmprow,data1.loc[data1['HoldingSharesLevel']=='800,001-1,000,000','percent'].values) # 14
    tmprow = np.append(tmprow,data1.loc[data1['HoldingSharesLevel']=='more than 1,000,001','percent'].values) # 15
    tmprow = np.append(tmprow,data1.loc[data1['HoldingSharesLevel']=='total','people'].values) # 16

    rs_sh.append(tmprow)
    rs_dt.append(dt)

  rs=pd.DataFrame(rs_sh,index=rs_dt,columns=['1','2','3','4','5','6','7','8','9','10','11','12','13','14','15','total_people'])
  return rs

def getPriceAndShareHolder(prod,st,en):

  data1=getDataFM(prod,st,en)
  data2=getFMShareHolder(prod,st,en)
  data2a=ShareHolderWeekly(data2)

  for i in range(1,data2a.shape[0]):
    tmpdata=data1.loc[(data1.index>data2a.index[i-1])&(data1.index<=data2a.index[i]),]
    if tmpdata.shape[0] == 0:
      continue
    data2a.loc[data2a.index[i],'open']=tmpdata.loc[tmpdata.index[0],'Open']
    data2a.loc[data2a.index[i],'high']=tmpdata.loc[:,'High'].max()
    data2a.loc[data2a.index[i],'low']=tmpdata.loc[:,'Low'].min()
    data2a.loc[data2a.index[i],'close']=tmpdata.loc[tmpdata.index[-1],'Close']
    data2a.loc[data2a.index,'volume']=tmpdata.loc[:,'Volume'].sum()

  data2a = data2a.dropna()
  return data2a

# 取得三大法人 FinMind資料來源
def getFMInstitutionalInvestors(prod, st, en):
    # 備份檔名
    bakfile = (
        f"{datapath}\\{prod}_{st}_{en}_TaiwanStockInstitutionalInvestorsBuySell.csv"
    )
    # 檢視是否有該檔案存在
    if os.path.exists(bakfile):
        # 取得檔案內容
        tmpdata = pd.read_csv(bakfile)
        tmpdata["date"] = pd.to_datetime(tmpdata["date"])
        tmpdata = tmpdata.set_index(tmpdata["date"])
    # 沒有的話就取API內容
    else:
        tmpdata = FM.taiwan_stock_institutional_investors(
            stock_id=prod, start_date=st, end_date=en
        )
        # 將日期設定為索引
        tmpdata["date"] = pd.to_datetime(tmpdata["date"])
        tmpdata = tmpdata.set_index(tmpdata["date"])
        tmpdata.drop("date", axis=1, inplace=True)
        # 將API內容寫入備份檔中
        tmpdata.to_csv(bakfile)
    # 回傳資料
    return tmpdata

# 將三大法人轉換為一日一筆
def InstInvestorsDaily(data):
    rs_sh, rs_dt = [], []
    # 依照日期去將資料做整合
    for dt in data.index.unique():
        # 撈出當天的所有資料
        data1 = data[data.index == dt]
        # 依照不同交易人類別
        tmprow = []
        f_b = data1.loc[data1["name"] == "Foreign_Investor", "buy"].values
        f_s = data1.loc[data1["name"] == "Foreign_Investor", "sell"].values
        fd_b = data1.loc[data1["name"] == "Foreign_Dealer_Self", "buy"].values
        fd_s = data1.loc[data1["name"] == "Foreign_Dealer_Self", "sell"].values
        i_b = data1.loc[data1["name"] == "Investment_Trust", "buy"].values
        i_s = data1.loc[data1["name"] == "Investment_Trust", "sell"].values
        d_b = data1.loc[data1["name"] == "Dealer", "buy"].values
        d_s = data1.loc[data1["name"] == "Dealer", "sell"].values
        ds_b = data1.loc[data1["name"] == "Dealer_self", "buy"].values
        ds_s = data1.loc[data1["name"] == "Dealer_self", "sell"].values
        dh_b = data1.loc[data1["name"] == "Dealer_Hedging", "buy"].values
        dh_s = data1.loc[data1["name"] == "Dealer_Hedging", "sell"].values
        # 如果為空 則填入該欄位為0
        if len(f_b) == 0:
            tmprow.append(0)
        else:
            tmprow.append(f_b[0])
        if len(f_s) == 0:
            tmprow.append(0)
        else:
            tmprow.append(f_s[0])
        if len(fd_b) == 0:
            tmprow.append(0)
        else:
            tmprow.append(fd_b[0])
        if len(fd_s) == 0:
            tmprow.append(0)
        else:
            tmprow.append(fd_s[0])
        if len(i_b) == 0:
            tmprow.append(0)
        else:
            tmprow.append(i_b[0])
        if len(i_s) == 0:
            tmprow.append(0)
        else:
            tmprow.append(i_s[0])
        if len(d_b) == 0:
            tmprow.append(0)
        else:
            tmprow.append(d_b[0])
        if len(d_s) == 0:
            tmprow.append(0)
        else:
            tmprow.append(d_s[0])
        if len(ds_b) == 0:
            tmprow.append(0)
        else:
            tmprow.append(ds_b[0])
        if len(ds_s) == 0:
            tmprow.append(0)
        else:
            tmprow.append(ds_s[0])
        if len(dh_b) == 0:
            tmprow.append(0)
        else:
            tmprow.append(dh_b[0])
        if len(dh_s) == 0:
            tmprow.append(0)
        else:
            tmprow.append(dh_s[0])
        # 將DF資料存放到容器
        rs_sh.append(tmprow)
        # 資料日期存成一個列表
        rs_dt.append(dt)
    # 將每日的資料整合
    rs = pd.DataFrame(
        rs_sh,
        index=rs_dt,
        columns=[
            "外陸資買進股數(不含外資自營商)",
            "外陸資賣出股數(不含外資自營商)",
            "外資自營商買進股數",
            "外資自營商賣出股數",
            "投信買進股數",
            "投信賣出股數",
            "自營商買進股數(自行買賣)_1",
            "自營商賣出股數(自行買賣)_1",
            "自營商買進股數(自行買賣)_2",
            "自營商賣出股數(自行買賣)_2",
            "自營商買進股數(避險)",
            "自營商賣出股數(避險)",
        ],
    )
    # 由於FinMind的，自營商類別有兩個，所以必須將這個了欄位合併，
    rs["自營商買進股數(自行買賣)"] = rs["自營商買進股數(自行買賣)_1"] + rs["自營商買進股數(自行買賣)_2"]
    rs["自營商賣出股數(自行買賣)"] = rs["自營商賣出股數(自行買賣)_1"] + rs["自營商賣出股數(自行買賣)_2"]
    rs.drop("自營商買進股數(自行買賣)_1", axis=1, inplace=True)
    rs.drop("自營商買進股數(自行買賣)_2", axis=1, inplace=True)
    rs.drop("自營商賣出股數(自行買賣)_1", axis=1, inplace=True)
    rs.drop("自營商賣出股數(自行買賣)_2", axis=1, inplace=True)
    return rs

# 取得K線與三大法人的集合資料 透過 FinMind
def getPriceAndInstInvest_FM(prod, st, en):
    # 取得還原股價
    data1 = getDataFM(prod, st, en)
    # 取得三大法人
    data2 = getFMInstitutionalInvestors(prod, st, en)
    # 轉換資料格式
    data2_1 = InstInvestorsDaily(data2)
    data3 = pd.concat([data1, data2_1], axis=1, join="inner")
    return data3

####TALIB

In [4]:
!pip install ta-lib-bin > log.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 357.1/357.1 kB 4.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
  error: subprocess-exited-with-error
  
  × Building wheel for Ta-Lib (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  ERROR: Failed building wheel for Ta-Lib
Failed to build Ta-Lib
ERROR: Could not build wheels for Ta-Lib, which is required to install pyproject.toml-based projects


In [5]:
import talib
from talib import SMA as func_sma
from talib.abstract import SMA as abs_sma
from talib.abstract import EMA

###**LINE** **Notify**

In [6]:
!pip install lineTool

  Preparing metadata (setup.py) ... done
  Created wheel for lineTool: filename=lineTool-1.0.3-py3-none-any.whl size=1439 sha256=7b978a2aa0a304a643db8e1d223285e95f6efdc779f1a025906d6fa55ee07d50
  Stored in directory: /root/.cache/pip/wheels/dd/ee/a8/c3dbde303ae7e30634142d641a482a7a683ba82554328e1941
Successfully built lineTool


In [7]:
import lineTool
# Line 推播
token="lppCbwUyaxN2tIBRz1R1gHd5vKpYCve4tyd5TTI501I" #我
# token="PsJ1IPfpwwAsZmQb0oi5U9Xgz3ZsLxIBtEqMyQ5dhS8" #00919股友社
# token ="TcKCmadWHAS7bE9AXdiRPVmyWY7d1wO4lmfc5TRJ2EZ" #梅家
def line_print(msg):
    print(msg)
    try :
        lineTool.lineNotify(token, msg)
    except:
        print('line notify 失效')
        pass

###**跟著大戶策略**

In [17]:
import datetime
# 取得回測資料
prod=int(input("請輸入要查詢的股票代碼:"))
st='2010-01-01'
en=datetime.datetime.now().strftime('%Y-%m-%d')
data=getPriceAndShareHolder(prod,st,en)

data['major'] = data['12'] + data['13'] + data['14'] + data['15'] #400張以上為大戶
data['major_ma'] = data.rolling(8)['major'].mean()
data['total_people_ma'] = data.rolling(8)['total_people'].mean()

position=0
# 當天訊號
signal=0

for i in range(data.shape[0]-1):
    c_time = data.index[i]
    c_high = data.loc[c_time,'high']
    c_close = data.loc[c_time,'close']
    c_major = data.loc[c_time,'major']
    c_major_ma = data.loc[c_time,'major_ma']
    c_total_people = data.loc[c_time,'total_people']
    c_total_people_ma = data.loc[c_time,'total_people_ma']

    n_time = data.index[i+1]
    n_open = data.loc[n_time,'open']

    # 進場程序
    if position == 0 :
      if c_major > c_major_ma and c_total_people > c_total_people_ma :
          position = 1
          signal = 3
      else :
          signal = 1
    # 出場程序
    elif position ==1 :
        # 出場邏輯
      if c_major < c_major_ma:
            position = 0
            signal = 4
      else:
            signal = 2

strategy_name='跟著大戶策略'
if signal==1:
    line_print('%s \n %s \n %s \n維持空手'%(strategy_name,prod,en))
elif signal==2:
    line_print('%s \n %s \n %s \n維持進場'%(strategy_name,prod,en))
elif signal==3:
    line_print('%s \n %s \n %s \n進場訊號'%(strategy_name,prod,en))
elif signal==4:
    line_print('%s \n %s \n %s \n出場訊號'%(strategy_name,prod,en))

請輸入要查詢的股票代碼:2401


2024-01-24 08:02:58.737 | INFO     | FinMind.data.finmind_api:get_data:125 - download TaiwanStockPrice, data_id: 2401
2024-01-24 08:03:08.515 | INFO     | FinMind.data.finmind_api:get_data:125 - download TaiwanStockHoldingSharesPer, data_id: 2401


跟著大戶策略 
 2401 
 2024-01-24 
維持空手


###**均線突破策略**




In [18]:
import datetime
# 取得回測資料
prod=int(input("請輸入要查詢的股票代碼:"))
st='2010-01-01'
en=datetime.datetime.now().strftime('%Y-%m-%d')
data = getDataFM1(prod,st,en)
# EMA(data,timeperiod=120)
data['ema'] = EMA(data,timeperiod=120)

movestoploss = 0.05

position = 0
trade = pd.DataFrame()

for i in range(data.shape[0]-1):
  c_time = data.index[i]
  c_high = data.loc[c_time,'high']
  c_close = data.loc[c_time,'close']
  c_ema = data.loc[c_time,'ema']

  n_time = data.index[i+1]
  n_open = data.loc[n_time,'open']

  if position == 0 :
    if c_close > c_ema*1.01 :
      position = 1
      signal = 3
    else:
      signal = 1
      # print(c_time,'觸發進場訊號 隔日進場', order_time,'進場價', order_price, '進場', order_unit,'單位')

  elif position == 1 :
    if c_close < c_ema * 0.995:
      position = 0
      signal = 4
    else:
      signal =2
      # print(c_time,'觸發出場訊號 隔日出場', order_time, '出場價', order_price)

strategy_name='突破均線策略'
if signal==1:
    line_print('%s \n %s \n %s \n維持空手'%(strategy_name,prod,en))
elif signal==2:
    line_print('%s \n %s \n %s \n維持進場'%(strategy_name,prod,en))
elif signal==3:
    line_print('%s \n %s \n %s \n進場訊號'%(strategy_name,prod,en))
elif signal==4:
    line_print('%s \n %s \n %s \n出場訊號'%(strategy_name,prod,en))

請輸入要查詢的股票代碼:2401


2024-01-24 08:03:32.792 | INFO     | FinMind.data.finmind_api:get_data:125 - download TaiwanStockPrice, data_id: 2401


突破均線策略 
 2401 
 2024-01-24 
維持進場


###**趨勢突破策略**

In [19]:
import datetime
# 取得回測資料
prod=int(input("請輸入要查詢的股票代碼:"))
st='2010-01-01'
en=datetime.datetime.now().strftime('%Y-%m-%d')
data = getDataFM(prod,st,en)
data['Ceil'] = data.rolling(3)['High'].max().shift()

movestoploss = 0.05

position = 0
sugnal= 0

for i in range(data.shape[0]-1):
  c_time = data.index[i]
  c_high = data.loc[c_time,'High']
  c_close = data.loc[c_time,'Close']
  c_ceil = data.loc[c_time,'Ceil']

  n_time = data.index[i+1]
  n_open = data.loc[n_time,'Open']

  if position == 0 :
    if c_close > c_ceil :
      position = 1
      order_i = i
      order_time = n_time
      order_price = n_open
      stoploss = order_price * (1 - movestoploss)
      # print(c_time,'觸發進場訊號 隔日進場', order_time,'進場價', order_price, '進場', order_unit,'單位')
      signal = 3
    else:
      signal = 1
  elif position == 1 :
    stoploss = max(stoploss,c_close * (1 - movestoploss))
    if c_close < stoploss:
      position = 0
      cover_time = n_time
      cover_price = n_open
      signal = 4
    else:
      signal = 2
      # print(c_time,'觸發出場訊號 隔日出場', order_time, '出場價', order_price)
strategy_name='趨勢突破策略'
if signal==1:
    line_print('%s \n %s \n %s \n維持空手'%(strategy_name,prod,en))
elif signal==2:
    line_print('%s \n %s \n %s \n維持進場'%(strategy_name,prod,en))
elif signal==3:
    line_print('%s \n %s \n %s \n進場訊號'%(strategy_name,prod,en))
elif signal==4:
    line_print('%s \n %s \n %s \n出場訊號'%(strategy_name,prod,en))

請輸入要查詢的股票代碼:2401


2024-01-24 08:03:42.266 | INFO     | FinMind.data.finmind_api:get_data:125 - download TaiwanStockPrice, data_id: 2401


趨勢突破策略 
 2401 
 2024-01-24 
維持進場


###**跟著外資策略**

In [25]:
import datetime
# 取得回測資料
prod=int(input("請輸入要查詢的股票代碼:"))
st='2010-01-01'
en=datetime.datetime.now().strftime('%Y-%m-%d')
data=getPriceAndInstInvest_FM(prod,st,en)

# 計算 外資平均買賣
data['F_day']=data['外陸資買進股數(不含外資自營商)']-data['外陸資賣出股數(不含外資自營商)']
data['F_month']=(data['F_day']).rolling(40).sum()
data['F_mean']=(data['F_month']).rolling(20).mean()
# 初始部位
position=0
# 當天訊號
signal=0

for i in range(data.shape[0]-1):
    # 取得策略會應用到的變數
    c_time=data.index[i]
    c_high=data.loc[c_time,'High']
    c_close=data.loc[c_time,'Close']
    c_F_month=data.loc[c_time,'F_month']
    c_F_mean=data.loc[c_time,'F_mean']
    # 取下一期資料做為進場資料
    n_time=data.index[i+1]
    n_open=data.loc[n_time,'Open']

    # 進場程序
    if position == 0 :
        if c_F_month > c_F_mean :
            position = 1
            signal = 3
        else :
            signal = 1
    # 出場程序
    elif position ==1 :
        # 出場邏輯
        if c_F_month < c_F_mean :
            position = 0
            signal = 4
        else:
            signal = 2

strategy_name='跟著外資策略'
if signal==1:
    line_print('%s \n %s \n %s \n維持空手'%(strategy_name,prod,en))
elif signal==2:
    line_print('%s \n %s \n %s \n維持進場'%(strategy_name,prod,en))
elif signal==3:
    line_print('%s \n %s \n %s \n進場訊號'%(strategy_name,prod,en))
elif signal==4:
    line_print('%s \n %s \n %s \n出場訊號'%(strategy_name,prod,en))

請輸入要查詢的股票代碼:2401


2024-01-24 08:06:10.601 | INFO     | FinMind.data.finmind_api:get_data:125 - download TaiwanStockPrice, data_id: 2401


跟著外資策略 
 2401 
 2024-01-24 
維持空手


###**反著散戶策略**

In [26]:
import datetime
# 取得回測資料
prod=int(input("請輸入要查詢的股票代碼:"))
st='2010-01-01'
en=datetime.datetime.now().strftime('%Y-%m-%d')
data=getPriceAndShareHolder(prod,st,en)

# 30張持有以下稱為散戶
data['minority']=data['1']+data['2']+data['3']+data['4']
# 算出過去的平均
data['minority_ma']=data.rolling(8)['minority'].mean()
data['ma']=data.rolling(4)['close'].mean()
data['total_people_ma']=data.rolling(8)['total_people'].mean()

position=0
# 當天訊號
signal=0

for i in range(data.shape[0]-1):
    # 取得策略會應用到的變數
    c_time=data.index[i]
    c_high=data.loc[c_time,'high']
    c_close=data.loc[c_time,'close']
    c_ma=data.loc[c_time,'ma']
    c_minority=data.loc[c_time,'minority']
    c_minority_ma=data.loc[c_time,'minority_ma']
    c_total_people=data.loc[c_time,'total_people']
    c_total_people_ma=data.loc[c_time,'total_people_ma']
    # 取下一期資料做為進場資料
    n_time=data.index[i+1]
    n_open=data.loc[n_time,'open']

    # 進場程序
    if position == 0 :
        if c_close > c_ma and c_minority < c_minority_ma :
          position = 1
          signal = 3
        else :
          signal = 1
    # 出場程序
    elif position ==1 :
        # 出場邏輯
        if c_minority > c_minority_ma:
            position = 0
            signal = 4
        else:
            signal = 2

strategy_name='反著散戶策略'
if signal==1:
    line_print('%s \n %s \n %s \n維持空手'%(strategy_name,prod,en))
elif signal==2:
    line_print('%s \n %s \n %s \n維持進場'%(strategy_name,prod,en))
elif signal==3:
    line_print('%s \n %s \n %s \n進場訊號'%(strategy_name,prod,en))
elif signal==4:
    line_print('%s \n %s \n %s \n出場訊號'%(strategy_name,prod,en))

請輸入要查詢的股票代碼:2401


2024-01-24 08:06:59.528 | INFO     | FinMind.data.finmind_api:get_data:125 - download TaiwanStockPrice, data_id: 2401


反著散戶策略 
 2401 
 2024-01-24 
出場訊號
